# Make index of retweet and mentions for each user

### Import data

In [1]:
import os
from google.colab import drive


# tweets folder contains all tweets
drive.mount('/content/drive')
data = '/content/drive/My Drive/covid_project/graph_files'
print(data)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/covid_project/graph_files


### make dictionary < user : connections >

In [3]:
import glob
import jsonlines
from urllib.parse import urlparse


users = {}
with jsonlines.open(data+"/sampled_tweets.jsonl") as infile:
    for line in infile:
        user = line['user']['name']

        # If it is retweet, we want to consider only the connection with the original user
        # this user is not interest by their mentions
        if 'retweeted_status' in line:
          connection = line['retweeted_status']['user']['name']

          if user in users:
            users[user].append(connection)
          else:
            users[user] = [connection]
        
        # If it simple tweet we want to extract all mentions
        else:
          connections = line['entities']['user_mentions']

          for connection in connections:
            if user in users:
              users[user].append(connection['name'])
            else:
              users[user] = [connection['name']]


In [4]:
print(f"Total users: {len(users)}")
#print(users)

Total users: 8687


The number of users in the dictionary is less than the number of users we sampled!  No worries, if we look at the statistics, this can be easily explained, considering that most users shared less than four tweets and in many cases, people do not mention anyone. 

In [5]:
import json
import jsonlines

file = data + '/retweet_mentions.jsonl'

with jsonlines.open(file, mode='w') as outfile:
    for entry in users:
      val = {'user':entry, 'connections':users[entry]}
      outfile.write(val)